# IBCo API Data Analysis Tutorial

This notebook demonstrates how to use the IBCo API for fiscal research and analysis.

**Prerequisites:**
```bash
pip install requests pandas matplotlib seaborn numpy
```

**Topics Covered:**
1. Connecting to the API
2. Fetching risk scores and trends
3. Analyzing pension data
4. Fiscal projections and scenario analysis
5. Data visualization
6. Exporting results

## Setup and Configuration

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Optional
from datetime import datetime

# Configure plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# API Configuration
BASE_URL = 'https://api.ibco-ca.us/api/v1'
# No authentication required - all data is public

## 1. API Client Setup

In [ ]:
class IBCoClient:
    """Simple client for IBCo API"""
    
    def __init__(self, base_url: str = BASE_URL):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({'Content-Type': 'application/json'})
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        """Make GET request to API"""
        url = f"{self.base_url}{endpoint}"
        response = self.session.get(url, params=params)
        response.raise_for_status()
        return response.json()
    
    def get_cities(self, state: Optional[str] = None) -> List[Dict]:
        params = {'state': state} if state else {}
        data = self.get('/cities', params)
        return data['cities']
    
    def get_current_risk_score(self, city_id: int) -> Dict:
        return self.get(f'/risk/cities/{city_id}/current')
    
    def get_risk_history(self, city_id: int, start_year: Optional[int] = None, 
                        end_year: Optional[int] = None) -> Dict:
        params = {}
        if start_year:
            params['start_year'] = start_year
        if end_year:
            params['end_year'] = end_year
        return self.get(f'/risk/cities/{city_id}/history', params)
    
    def get_fiscal_cliff(self, city_id: int, scenario: str = 'base') -> Dict:
        return self.get(f'/projections/cities/{city_id}/fiscal-cliff', 
                       {'scenario': scenario})
    
    def get_pension_status(self, city_id: int, fiscal_year: int) -> Dict:
        return self.get(f'/pensions/cities/{city_id}/status', 
                       {'fiscal_year': fiscal_year})

# Initialize client
client = IBCoClient()
print("✓ API client initialized")

## 2. Exploring Available Cities

In [ ]:
# Get all California cities
cities = client.get_cities(state='CA')
cities_df = pd.DataFrame(cities)

print(f"Found {len(cities_df)} cities in California\n")
print(cities_df[['id', 'name', 'state', 'population']].head(10))

## 3. Analyzing Risk Scores

In [ ]:
# Get current risk score for Vallejo
CITY_ID = 1  # Vallejo
risk_score = client.get_current_risk_score(CITY_ID)

print(f"City: {risk_score['city_name']}")
print(f"Fiscal Year: {risk_score['fiscal_year']}")
print(f"Overall Score: {risk_score['overall_score']:.1f}/100 ({risk_score['risk_level']})")
print(f"Data Completeness: {risk_score['data_completeness_percent']:.1f}%\n")
print("Category Scores:")
for category, score in risk_score['category_scores'].items():
    print(f"  {category:25s}: {score:.1f}/100")

In [ ]:
# Visualize category breakdown
categories = list(risk_score['category_scores'].keys())
scores = list(risk_score['category_scores'].values())

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#D32F2F' if s >= 75 else '#FF5722' if s >= 50 else '#FFC107' if s >= 25 else '#4CAF50' 
          for s in scores]
bars = ax.barh(categories, scores, color=colors)
ax.set_xlabel('Risk Score (0-100)', fontsize=12)
ax.set_title(f"{risk_score['city_name']} Risk Score Breakdown - FY{risk_score['fiscal_year']}", 
             fontsize=14, fontweight='bold')
ax.axvline(x=25, color='green', linestyle='--', alpha=0.3, label='Low threshold')
ax.axvline(x=50, color='orange', linestyle='--', alpha=0.3, label='Moderate threshold')
ax.axvline(x=75, color='red', linestyle='--', alpha=0.3, label='High threshold')
ax.set_xlim(0, 100)
ax.legend()

# Add score labels
for i, (bar, score) in enumerate(zip(bars, scores)):
    ax.text(score + 2, bar.get_y() + bar.get_height()/2, f'{score:.1f}', 
            va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Historical Risk Trends

In [ ]:
# Get risk history
history = client.get_risk_history(CITY_ID, start_year=2015)

# Convert to DataFrame
history_df = pd.DataFrame(history['risk_scores'])
print(f"Trend: {history['trend']}")
print(f"\nHistorical scores for {history['city_name']}:")
print(history_df[['fiscal_year', 'overall_score', 'risk_level']])

In [ ]:
# Plot risk score trend
fig, ax = plt.subplots(figsize=(12, 6))

# Plot overall score
ax.plot(history_df['fiscal_year'], history_df['overall_score'], 
        marker='o', linewidth=2, markersize=8, label='Overall Score', color='#2196F3')

# Add risk level zones
ax.axhspan(0, 25, alpha=0.1, color='green', label='Low Risk')
ax.axhspan(25, 50, alpha=0.1, color='yellow', label='Moderate Risk')
ax.axhspan(50, 75, alpha=0.1, color='orange', label='High Risk')
ax.axhspan(75, 100, alpha=0.1, color='red', label='Severe Risk')

ax.set_xlabel('Fiscal Year', fontsize=12)
ax.set_ylabel('Risk Score', fontsize=12)
ax.set_title(f"{history['city_name']} Risk Score Trend ({history['trend']})", 
             fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Pension Analysis

In [ ]:
# Get pension status
pensions = client.get_pension_status(CITY_ID, fiscal_year=2024)

print(f"Pension Status - {pensions['city_name']} FY{pensions['fiscal_year']}")
print(f"Average Funded Ratio: {pensions['avg_funded_ratio']*100:.1f}%")
print(f"Total Unfunded Liability: ${pensions['total_ual']:,.0f}")
print(f"Total Contributions: ${pensions['total_contributions']:,.0f}")
print(f"\nPension Plans:")

plans_df = pd.DataFrame(pensions['plans'])
plans_df['funded_ratio_pct'] = plans_df['funded_ratio'] * 100
print(plans_df[['plan_name', 'funded_ratio_pct', 'ual', 'employer_contribution']])

In [ ]:
# Visualize pension plan funded ratios
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Funded Ratios
colors_fr = ['#D32F2F' if fr < 0.70 else '#FFC107' if fr < 0.80 else '#4CAF50' 
             for fr in plans_df['funded_ratio']]
bars1 = ax1.barh(plans_df['plan_name'], plans_df['funded_ratio_pct'], color=colors_fr)
ax1.set_xlabel('Funded Ratio (%)', fontsize=11)
ax1.set_title('Pension Plan Funded Ratios', fontsize=12, fontweight='bold')
ax1.axvline(x=70, color='red', linestyle='--', alpha=0.5, label='Critical (70%)')
ax1.axvline(x=80, color='orange', linestyle='--', alpha=0.5, label='Concerning (80%)')
ax1.legend()

# Add percentage labels
for bar, pct in zip(bars1, plans_df['funded_ratio_pct']):
    ax1.text(pct + 1, bar.get_y() + bar.get_height()/2, f'{pct:.1f}%', 
             va='center', fontweight='bold', fontsize=9)

# Plot 2: Unfunded Liability Distribution
ax2.pie(plans_df['ual'], labels=plans_df['plan_name'], autopct='%1.1f%%',
        startangle=90, colors=sns.color_palette('Set3', len(plans_df)))
ax2.set_title(f'UAL Distribution\nTotal: ${pensions["total_ual"]:,.0f}', 
              fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Fiscal Cliff Analysis

In [ ]:
# Analyze fiscal cliff under different scenarios
scenarios = ['base', 'optimistic', 'pessimistic']
cliff_data = {}

for scenario in scenarios:
    cliff = client.get_fiscal_cliff(CITY_ID, scenario=scenario)
    cliff_data[scenario] = cliff
    
    print(f"\n{scenario.upper()} SCENARIO:")
    if cliff['has_fiscal_cliff']:
        print(f"  ⚠ Fiscal cliff in FY{cliff['fiscal_cliff_year']} ({cliff['years_until_cliff']} years)")
        print(f"  Cumulative deficit: ${cliff['cumulative_deficit_at_cliff']:,.0f}")
        if cliff.get('revenue_increase_needed_percent'):
            print(f"  Revenue increase needed: {cliff['revenue_increase_needed_percent']:.1f}%")
        if cliff.get('expenditure_decrease_needed_percent'):
            print(f"  OR Expenditure decrease needed: {cliff['expenditure_decrease_needed_percent']:.1f}%")
    else:
        print("  ✓ No fiscal cliff projected")

In [ ]:
# Compare scenarios
scenario_comparison = []
for scenario, data in cliff_data.items():
    scenario_comparison.append({
        'Scenario': scenario.capitalize(),
        'Has Cliff': 'Yes' if data['has_fiscal_cliff'] else 'No',
        'Cliff Year': data.get('fiscal_cliff_year', 'N/A'),
        'Years Until': data.get('years_until_cliff', 'N/A'),
        'Deficit at Cliff': data.get('cumulative_deficit_at_cliff', 0)
    })

comparison_df = pd.DataFrame(scenario_comparison)
print("\nScenario Comparison:")
print(comparison_df)

In [ ]:
# Visualize scenario comparison
fig, ax = plt.subplots(figsize=(10, 6))

cliff_years = [data.get('years_until_cliff', 0) if data['has_fiscal_cliff'] else 0 
               for data in cliff_data.values()]
colors = ['#4CAF50' if y == 0 else '#FFC107' if y > 5 else '#D32F2F' 
          for y in cliff_years]

bars = ax.bar(scenarios, cliff_years, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Years Until Fiscal Cliff', fontsize=12)
ax.set_xlabel('Scenario', fontsize=12)
ax.set_title('Fiscal Cliff Timeline by Scenario', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(cliff_years) + 2 if cliff_years else 10)

# Add value labels
for bar, years, scenario in zip(bars, cliff_years, scenarios):
    label = f'{years} years' if years > 0 else 'No cliff'
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, label,
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Comprehensive Dashboard

In [ ]:
# Create a comprehensive fiscal health dashboard
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Overall Risk Score Gauge
ax1 = fig.add_subplot(gs[0, 0])
overall = risk_score['overall_score']
colors_gauge = ['#4CAF50', '#FFC107', '#FF5722', '#D32F2F']
thresholds = [25, 50, 75, 100]
for i, (threshold, color) in enumerate(zip(thresholds, colors_gauge)):
    start = thresholds[i-1] if i > 0 else 0
    ax1.barh(0, threshold - start, left=start, height=0.5, color=color, alpha=0.5)
ax1.plot([overall, overall], [-0.25, 0.25], 'k-', linewidth=3)
ax1.set_xlim(0, 100)
ax1.set_ylim(-0.5, 0.5)
ax1.set_xlabel('Risk Score')
ax1.set_title(f'Overall Risk: {overall:.1f} ({risk_score["risk_level"]})', fontweight='bold')
ax1.set_yticks([])

# 2. Category Radar Chart
ax2 = fig.add_subplot(gs[0, 1:], projection='polar')
categories_radar = list(risk_score['category_scores'].keys())
values = list(risk_score['category_scores'].values())
angles = np.linspace(0, 2 * np.pi, len(categories_radar), endpoint=False).tolist()
values += values[:1]
angles += angles[:1]
ax2.plot(angles, values, 'o-', linewidth=2, color='#2196F3')
ax2.fill(angles, values, alpha=0.25, color='#2196F3')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels([c[:15] for c in categories_radar], size=8)
ax2.set_ylim(0, 100)
ax2.set_title('Risk Category Breakdown', fontweight='bold', pad=20)
ax2.grid(True)

# 3. Risk Trend
ax3 = fig.add_subplot(gs[1, :])
ax3.plot(history_df['fiscal_year'], history_df['overall_score'], 
         marker='o', linewidth=2, markersize=6, color='#2196F3')
ax3.axhspan(0, 25, alpha=0.1, color='green')
ax3.axhspan(25, 50, alpha=0.1, color='yellow')
ax3.axhspan(50, 75, alpha=0.1, color='orange')
ax3.axhspan(75, 100, alpha=0.1, color='red')
ax3.set_xlabel('Fiscal Year')
ax3.set_ylabel('Risk Score')
ax3.set_title('Historical Risk Trend', fontweight='bold')
ax3.set_ylim(0, 100)
ax3.grid(True, alpha=0.3)

# 4. Pension Funded Ratios
ax4 = fig.add_subplot(gs[2, 0])
colors_pension = ['#D32F2F' if fr < 0.70 else '#FFC107' if fr < 0.80 else '#4CAF50' 
                  for fr in plans_df['funded_ratio']]
ax4.barh(range(len(plans_df)), plans_df['funded_ratio_pct'], color=colors_pension)
ax4.set_yticks(range(len(plans_df)))
ax4.set_yticklabels([p[:20] for p in plans_df['plan_name']], size=8)
ax4.set_xlabel('Funded Ratio (%)')
ax4.set_title('Pension Funded Ratios', fontweight='bold')
ax4.axvline(x=80, color='orange', linestyle='--', alpha=0.5)

# 5. Fiscal Cliff Timeline
ax5 = fig.add_subplot(gs[2, 1])
cliff_years_plot = [data.get('years_until_cliff', 0) if data['has_fiscal_cliff'] else 0 
                    for data in cliff_data.values()]
colors_cliff = ['#4CAF50' if y == 0 else '#FFC107' if y > 5 else '#D32F2F' 
                for y in cliff_years_plot]
ax5.bar(scenarios, cliff_years_plot, color=colors_cliff, alpha=0.7)
ax5.set_ylabel('Years Until Cliff')
ax5.set_title('Fiscal Cliff Scenarios', fontweight='bold')

# 6. Key Metrics Summary
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
metrics_text = f"""
KEY METRICS SUMMARY
━━━━━━━━━━━━━━━━━━━━━━

Risk Score: {risk_score['overall_score']:.1f}/100
Risk Level: {risk_score['risk_level'].upper()}

Pension Funded: {pensions['avg_funded_ratio']*100:.1f}%
Total UAL: ${pensions['total_ual']/1e6:.1f}M

Base Scenario:
  Cliff: {'Yes' if cliff_data['base']['has_fiscal_cliff'] else 'No'}
  Years: {cliff_data['base'].get('years_until_cliff', 'N/A')}

Data Quality: {risk_score['data_completeness_percent']:.0f}%
"""
ax6.text(0.1, 0.5, metrics_text, fontsize=10, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', 
         facecolor='wheat', alpha=0.3))

fig.suptitle(f"{risk_score['city_name']} Fiscal Health Dashboard - FY{risk_score['fiscal_year']}", 
             fontsize=16, fontweight='bold')
plt.show()

## 8. Export Results

In [ ]:
# Export data to CSV files for further analysis
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Export risk history
history_df.to_csv(f'risk_history_{timestamp}.csv', index=False)
print(f"✓ Exported risk_history_{timestamp}.csv")

# Export pension data
plans_df.to_csv(f'pension_plans_{timestamp}.csv', index=False)
print(f"✓ Exported pension_plans_{timestamp}.csv")

# Export scenario comparison
comparison_df.to_csv(f'scenario_comparison_{timestamp}.csv', index=False)
print(f"✓ Exported scenario_comparison_{timestamp}.csv")

# Export current risk score
risk_score_df = pd.DataFrame([{
    'city_name': risk_score['city_name'],
    'fiscal_year': risk_score['fiscal_year'],
    'overall_score': risk_score['overall_score'],
    'risk_level': risk_score['risk_level'],
    'data_completeness': risk_score['data_completeness_percent'],
    **risk_score['category_scores']
}])
risk_score_df.to_csv(f'current_risk_score_{timestamp}.csv', index=False)
print(f"✓ Exported current_risk_score_{timestamp}.csv")

## 9. Advanced Analysis: Multi-City Comparison (if data available)

In [ ]:
# Compare multiple cities (when data is available)
# This is a template for future use when more cities have data

def compare_cities(city_ids: List[int], fiscal_year: int = 2024):
    """Compare fiscal health across multiple cities"""
    comparison = []
    
    for city_id in city_ids:
        try:
            risk = client.get_current_risk_score(city_id)
            pension = client.get_pension_status(city_id, fiscal_year)
            
            comparison.append({
                'city_name': risk['city_name'],
                'overall_risk': risk['overall_score'],
                'risk_level': risk['risk_level'],
                'pension_funded': pension['avg_funded_ratio'] * 100,
                'total_ual': pension['total_ual']
            })
        except Exception as e:
            print(f"Could not fetch data for city {city_id}: {e}")
    
    return pd.DataFrame(comparison)

# Example usage (uncomment when more cities have data):
# cities_to_compare = [1, 2, 3]  # Vallejo and others
# comparison_df = compare_cities(cities_to_compare)
# print(comparison_df)

## Conclusion

This notebook demonstrated:
- ✓ Connecting to the IBCo API
- ✓ Fetching and analyzing risk scores
- ✓ Exploring pension data and trends
- ✓ Comparing fiscal projection scenarios
- ✓ Creating visualizations
- ✓ Exporting data for further analysis

**Next Steps:**
1. Customize the analysis for your research questions
2. Explore additional endpoints (see API_USAGE_GUIDE.md)
3. Combine with external datasets
4. Develop predictive models

**Resources:**
- API Documentation: https://api.ibco-ca.us/docs
- API Usage Guide: docs/API_USAGE_GUIDE.md
- Developer Guide: docs/DEVELOPER_GUIDE.md
- Code Examples: examples/

**Rate Limits:**
- Public tier: 100 requests/hour
- Researcher tier: 1000 requests/hour (contact for API token)

**Citation:**
If you use this data in research, please cite:
```
IBCo Vallejo Console (2024). California Municipal Fiscal Risk Database. 
https://ibco-ca.us
```